In [37]:
import pygsheets
import difflib
import re
import pandas as pd
import numpy as np


In [2]:
def client_auth(use_pyg=True):
    # authorization
    # using pygsheets
    if use_pyg:
        gc = pygsheets.authorize(service_file="client_secret.json")
        return gc
    else:
        scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
        creds = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope)
        client = gspread.authorize(creds)
        return client

In [3]:
def fix_names(name):
    return name.split(' ')

def fix_depts(dept):
    return dept

In [4]:
gc=client_auth()

In [6]:
senate_sh = gc.open('AC Senate Data 2023')
senate_wk = senate_sh[1]
senate_df = senate_wk.get_as_df(header=True)
senate_df

,senate order,Department_Classes,Department_Senate,Dept Code,Dept_Cleaned,Class (on Senate),Course Number,course order (for sorting),Instructors,Co-instructor?,# instructors,Cross-listed?,UC Extension?,Deprecated,Course Name Change,Notes,Internal Notes
0,1,AFRICAN AMERICAN STUDIES,AFRICAN AMERICAN STUDIES,AFRICAM,AFRICAM,17AC,17AC,17.0,VeVe Clark,FALSE,1,TRUE,FALSE,FALSE,FALSE,Cross-listed with History 17A,
1,2,AFRICAN AMERICAN STUDIES,AFRICAN AMERICAN STUDIES,AFRICAM,AFRICAM,C20AC,C20AC,20.0,Saru Jayaraman and Michael Cohen,TRUE,2,TRUE,FALSE,FALSE,FALSE,Cross-listed with Public Policy C20AC,
2,3,AFRICAN AMERICAN STUDIES,AFRICAN AMERICAN STUDIES,AFRICAM,AFRICAM,25AC,25AC,25.0,William Banks,FALSE,1,FALSE,FALSE,FALSE,TRUE,previously AfricAm 25,
3,7,AFRICAN AMERICAN STUDIES,AFRICAN AMERICAN STUDIES,AFRICAM,AFRICAM,27AC,27AC,27.0,Leigh Raiford,FALSE,1,FALSE,FALSE,FALSE,FALSE,,
4,6,AFRICAN AMERICAN STUDIES,AFRICAN AMERICAN STUDIES,AFRICAM,AFRICAM,27AC,27AC,27.0,Michael Cohen,FALSE,1,FALSE,FALSE,FALSE,FALSE,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1269,1123,cannot find on classes,RELIGIOUS STUDIES,,,172AC,172AC,172.0,Luis Leon,FALSE,1,FALSE,FALSE,FALSE,FALSE,,
1270,1124,cannot find on classes,RELIGIOUS STUDIES,,,173AC,173AC,173.0,Purushottama Bilimoria,FALSE,1,TRUE,FALSE,FALSE,FALSE,Cross-listed with African American Studies 173AC,
1271,,,,,,,,#N/A,,,,,,,,,
1272,,,,,,,,#N/A,,,,,,,,,


In [42]:
semester_sh = gc.open("AC Classes List")
sheet_names = [s.title for s in semester_sh.worksheets()]
print(sheet_names)
semester_wk = semester_sh.worksheet_by_title("2023 Fall")
latest_classes = semester_wk.get_as_df(header=True)
latest_classes

['Copy of Spring 2020 (as of 11/13/2019)', "Approved['Spring 2020']", "Not Approved['Spring 2020']", '2020 Summer Session', "Not Approved['2020 Summer Session']", "Copy of Not Approved['2020 Summer Session']", "Approved['2020 Summer Session']", '2020 Fall', '2017 Spring', '2017 Summer-6W1', '2017 Fall', '2018 Spring', '2018 Summer Session', '2018 Fall', 'Spring 2019', '2019 Summer Session', '2019 Fall', 'Spring 2020', '2021 Summer Session', '2021 Fall', 'Spring 2021', 'Spring 2022', '2022 Summer Session', '2022 Fall', 'Spring 2023', '2023 Summer Session_ALL', '2023 Fall', '2023 Summer Session']


,Department,Department_FULL,Instruction Mode,Classes Links,Course Number,Instructor 1,Instructor 2,Total Open Seats,Enrolled,Waitlisted,Capacity,Waitlist Max,Open Reserved Seats,Also offered as
0,AFRICAM,AFRICAN AMERICAN STUDIES,In-Person Instruction,https://classes.berkeley.edu//content/2023-fal...,27AC,Michael M Cohen,NaN,2,198,29,200,30,None Reserved,NaN
1,AFRICAM,AFRICAN AMERICAN STUDIES,In-Person Instruction,https://classes.berkeley.edu//content/2023-fal...,C133A,Travis Bristol,NaN,0,23,4,23,5,None Reserved,EDUC C181
2,AMERSTD,UNDERGRADUATE INTERDISCIPLINARY STUDIES,In-Person Instruction,https://classes.berkeley.edu//content/2023-fal...,101AC,Sarah Erina Gold McBride,NaN,20,30,20,50,20,"21 reserved for American Studies Majors,",NaN
3,AMERSTD,UNDERGRADUATE INTERDISCIPLINARY STUDIES,In-Person Instruction,https://classes.berkeley.edu//content/2023-fal...,102AC,Sarah Erina Gold McBride,NaN,19,31,20,50,20,"21 reserved for American Studies Majors,",NaN
4,ANTHRO,ANTHROPOLOGY,Online,https://classes.berkeley.edu//content/2023-fal...,2AC,Christopher Lowman,NaN,9,354,36,363,37,"17 seats reserved for Anthropology Majors, 17 ...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,XGWS,UC BERKELEY EXTENSION,Online,https://classes.berkeley.edu//content/2023-fal...,50AC,NA,NaN,48,0,0,48,0,None Reserved,NaN
62,XHISTOR,UC BERKELEY EXTENSION,In-Person Instruction,https://classes.berkeley.edu//content/2023-fal...,7B,Samantha H Gervase,NaN,0,48,3,48,8,None Reserved,NaN
63,XHISTOR,UC BERKELEY EXTENSION,In-Person Instruction,https://classes.berkeley.edu//content/2023-fal...,7B,Samantha H Gervase,NaN,0,24,1,24,4,None Reserved,NaN
64,XMUSIC,UC BERKELEY EXTENSION,In-Person Instruction,https://classes.berkeley.edu//content/2023-fal...,26AC,Francesca Rivera,NaN,28,20,0,48,8,None Reserved,NaN


In [43]:
# Now check if the names are in the Google Spreadsheet
'''for name, course in names:
    closest_match = difflib.get_close_matches(name, spreadsheet_values, n=1, cutoff=0.6)
    if closest_match:
        print(f'{name} who teaches {course} is probably in the spreadsheet as {closest_match[0]}.')
    else:
        print(f'{name} who teaches {course} is not in the spreadsheet.')'''

"for name, course in names:\n    closest_match = difflib.get_close_matches(name, spreadsheet_values, n=1, cutoff=0.6)\n    if closest_match:\n        print(f'{name} who teaches {course} is probably in the spreadsheet as {closest_match[0]}.')\n    else:\n        print(f'{name} who teaches {course} is not in the spreadsheet.')"

In [45]:
instructor_cols = [c for c in latest_classes.columns if 'Instructor' in c]
instructor_cols

['Instructor 1', 'Instructor 2']

In [269]:
# loop through every course in the current semester
    # first filter senate by department and number
    # match first and last names with a 90% threshhold
    # if there is a match, then mark as approved
    # else: flag for manual checking

for i, row in latest_classes.iterrows():
    dept = row['Department']
    code = row['Course Number']
    instructors = []
    print(row.index)
    j = 1
    for colname in instructor_cols:
        if pd.notnull(row[colname]):
            instructors.append(row[colname].lower())
        else:
            break
    
     
    print(dept, code, instructors)

    approved = senate_df.loc[senate_df['Dept_Cleaned'].isin([dept])  &  senate_df['Course Number'].isin([code])
                             , ['Department_Classes', 'Dept_Cleaned', 'Course Number', 'Instructors', 'Co-instructor?', '# instructors', 'Cross-listed?']]
    print(approved)
    #
    approved['Instructors'] = approved['Instructors'].str.lower()


    print(row['Also offered as'])

    if len(instructors) == 1:
        status = check_one_name(instructors[0], approved)
    
    else:
        # co instructors!!!
        check_co_names(instructors, approved)

    break


Index(['Department', 'Department_FULL', 'Instruction Mode', 'Classes Links',
       'Course Number', 'Instructor 1', 'Instructor 2', 'Total Open Seats',
       'Enrolled', 'Waitlisted', 'Capacity', 'Waitlist Max',
       'Open Reserved Seats', 'Also offered as'],
      dtype='object')
AFRICAM 27AC ['michael m cohen']
          Department_Classes Dept_Cleaned Course Number     Instructors  \
3  AFRICAN AMERICAN STUDIES       AFRICAM          27AC   Leigh Raiford   
4  AFRICAN AMERICAN STUDIES       AFRICAM          27AC   Michael Cohen   
5  AFRICAN AMERICAN STUDIES       AFRICAM          27AC   Percy Hintzen   
6  AFRICAN AMERICAN STUDIES       AFRICAM          27AC  Rose Wilkerson   
7  AFRICAN AMERICAN STUDIES       AFRICAM          27AC      Tommy Lott   

  Co-instructor? # instructors Cross-listed?  
3          FALSE             1         FALSE  
4          FALSE             1         FALSE  
5          FALSE             1         FALSE  
6          FALSE             1         FAL

In [247]:
approved = senate_df.loc[senate_df['Dept_Cleaned'].isin(["AMERSTD"])  &  senate_df['Course Number'].isin(['101AC'])
                             , ['Department_Classes', 'Dept_Cleaned', 'Course Number', 'Instructors', 'Co-instructor?', '# instructors', 'Cross-listed?']]
approved

,Department_Classes,Dept_Cleaned,Course Number,Instructors,Co-instructor?,# instructors,Cross-listed?
43,UNDERGRADUATE INTERDISCIPLINARY STUDIES,AMERSTD,101AC,David Henkin,FALSE,1,FALSE
44,UNDERGRADUATE INTERDISCIPLINARY STUDIES,AMERSTD,101AC,Donald McQuade,FALSE,1,FALSE
45,UNDERGRADUATE INTERDISCIPLINARY STUDIES,AMERSTD,101AC,Kathleen Moran,FALSE,1,FALSE
46,UNDERGRADUATE INTERDISCIPLINARY STUDIES,AMERSTD,101AC,Mark Brilliant,FALSE,1,FALSE
47,UNDERGRADUATE INTERDISCIPLINARY STUDIES,AMERSTD,101AC,Michael Cohen,FALSE,1,FALSE
48,UNDERGRADUATE INTERDISCIPLINARY STUDIES,AMERSTD,101AC,Robert Nelson,FALSE,1,FALSE
49,UNDERGRADUATE INTERDISCIPLINARY STUDIES,AMERSTD,101AC,Sarah Gold McBride,FALSE,1,FALSE
50,UNDERGRADUATE INTERDISCIPLINARY STUDIES,AMERSTD,101AC,Waldo Martin,FALSE,1,FALSE


In [248]:
approval_status = []
if len(instructors) == 1:
    # single instructor
    # just start checking
    print()

In [281]:

def check_one_name(name, roster):
    print(roster['Instructors'])

    closest_match = difflib.get_close_matches(name, roster['Instructors'], n=1, cutoff=0.9)
    print('full match:',  closest_match)
    if closest_match:
        print(f'{name} who teaches {roster["Dept_Cleaned"].values[0]} {roster["Course Number"].values[0]} is probably in the spreadsheet as {closest_match[0]}.')
        #approval_status.append('Approved')
        
        return [ "Match" if inst == closest_match[0] else "No Match" for inst in roster['Instructors']]
    else:
        approval_status = []
        
        #approval_status.append('Marked as Not Approved by Code')
        # check by splitting first, middle, last:
        n = [w for w in name.split(' ') if len(w) > 1]
        for i, apr_name in enumerate(roster['Instructors'].values):
            a = [w for w in apr_name.split(' ') if len(w) > 1]
            score = 0
            if len(n) == len(a):
                for i in range(len(n)):
                    #print('same len', a[i], n[i], i, ":", score)
                    score += difflib.SequenceMatcher(None, n[i], a[i]).ratio() 
                    #score = difflib.SequenceMatcher(None, n[len(n)-1], a[len(a)-1]).ratio()
                
                score /= len(n) # average score -- A higher SCORE IS BETTER (smaller differences!)
                if score > .8:
                    approval_status.append('Match')
            
            elif len(n) > len(a): # classes.berkeley.edu display name is longer; likely theres a middle name or something
                # TODO: loop through a list and match first name and end to middle.
                # if first names don't match with a score above 80%, then its prob not a match:
                #print(n, a)
                if difflib.SequenceMatcher(None, n[0], a[0]).ratio() > .7 and difflib.SequenceMatcher(None, n[len(n)-1], a[len(a)-1]).ratio() > .7:
                        #print("FIRST AND LAST MATCH w .7")
                        #print('lw err', n[len(n)-1], a[len(a)-1])
                        #print('lw_err', difflib.SequenceMatcher(None, n[len(n)-1], a[len(a)-1]).ratio())
                        n_mid = ' '.join(n[1:len(n)-1]).strip()
                        a_mid = ' '.join(a[1:len(a)-1]).strip()
                        middle_score = difflib.SequenceMatcher(None, n_mid, a_mid).ratio()
                        if n_mid == '' or a_mid == '':
                            #print("NO MATCH: Should Doublecheck: Middle name differences.")
                            approval_status.append('Unsure')
                        elif middle_score > .3:
                            #print ("match?:", n[0] + ' ' + n[len(n)-1], a[0] + ' ' + a[len(a)-1])
                            approval_status.append('Match')
                else:
                    approval_status.append('No Match')
                
                    # simplified: match the middle words. see how diff it is.
                    
                
                #print(f'{name} who teaches {roster["Dept_Cleaned"].values[0]} {roster["Course Number"].values[0]} doesn\'t match {apr_name}.')
                
            else: # senate name is longer than the display name ORR its a co-instructor approval. (where there would be like multiple names)
                #print(roster['Co-instructor?'])
                if roster['Co-instructor?'].values[i] == True:
                    
                    #print(f'{name} who teaches {roster["Dept_Cleaned"].values[0]} {roster["Course Number"].values[0]} doesn\'t match {apr_name} by co-instructor rule')
                    approval_status.append('No Match')
                
                else: # rare case where senate name is longer than display name -- likely a middle name or secondary name somewhere
                    # if first names don't match with a score above 80%, then its prob not a match:
                    if difflib.SequenceMatcher(None, n[0], a[0]).ratio() > .7 and difflib.SequenceMatcher(None, n[len(n)-1], a[len(a)-1]).ratio() > .7:
                        #print("FIRST AND LAST MATCH w .7")
                        #print('lw err', n[len(n)-1], a[len(a)-1])
                        #print('lw_err', difflib.SequenceMatcher(None, n[len(n)-1], a[len(a)-1]).ratio())
                        n_mid = ' '.join(n[1:len(n)-1]).strip()
                        a_mid = ' '.join(a[1:len(a)-1]).strip()
                        middle_score = difflib.SequenceMatcher(None, n_mid, a_mid).ratio()
                        if n_mid == '' or a_mid == '':
                            #print("NO MATCH: Should Doublecheck: Middle name differences.")
                            approval_status.append('Unsure')
                        elif middle_score > .3:
                            #print ("match?:", n[0] + ' ' + n[len(n)-1], a[0] + ' ' + a[len(a)-1])
                            approval_status.append('Match')
                    else:
                        approval_status.append('No Match')
    #print(approval_status)
    #print('Match' in approval_status)
    if 'Match' in approval_status:
        i = approval_status.index('Match')
        print(f'{name} who teaches {roster["Dept_Cleaned"].values[0]} {roster["Course Number"].values[0]} is probably APPROVED as {approved["Instructors"].values[i]}.')
    elif 'Unsure' in approval_status:
        i = approval_status.index('Unsure')
        print(f'{name} who teaches {roster["Dept_Cleaned"].values[0]} {roster["Course Number"].values[0]} is probably APPROVED as {approved["Instructors"].values[i]}. Please double check.')
    else:
        print(f'{name} who teaches {roster["Dept_Cleaned"].values[0]} {roster["Course Number"].values[0]} is not in the spreadsheet.')
    return approval_status

    

In [271]:
#def check_co_names(names, roster):
approved

,Department_Classes,Dept_Cleaned,Course Number,Instructors,Co-instructor?,# instructors,Cross-listed?
3,AFRICAN AMERICAN STUDIES,AFRICAM,27AC,leigh raiford,FALSE,1,FALSE
4,AFRICAN AMERICAN STUDIES,AFRICAM,27AC,michael cohen,FALSE,1,FALSE
5,AFRICAN AMERICAN STUDIES,AFRICAM,27AC,percy hintzen,FALSE,1,FALSE
6,AFRICAN AMERICAN STUDIES,AFRICAM,27AC,rose wilkerson,FALSE,1,FALSE
7,AFRICAN AMERICAN STUDIES,AFRICAM,27AC,tommy lott,FALSE,1,FALSE


In [283]:
check_one_name("leigh raiford", approved)



3     leigh raiford
4     michael cohen
5     percy hintzen
6    rose wilkerson
7        tommy lott
Name: Instructors, dtype: object
full match: ['leigh raiford']
leigh raiford who teaches AFRICAM 27AC is probably in the spreadsheet as leigh raiford.


['Match', 'No Match', 'No Match', 'No Match', 'No Match']